In [1]:
import os
import sys

In [2]:
current_dir = os.getcwd()
if current_dir.endswith('basic'):
    project_root = os.path.dirname(current_dir)
else:
    project_root = os.path.join(current_dir, '..')
print(project_root)
if project_root not in sys.path:
    print(f"Adding {project_root} to sys.path")
    sys.path.insert(0, project_root)

/home/iwaniukooo/Documents/Projects/wb2-llm/llm-workshop/advanced/..
Adding /home/iwaniukooo/Documents/Projects/wb2-llm/llm-workshop/advanced/.. to sys.path


In [3]:
from basic.llm_model import LLMModel
# from llm_model import LLMModel
from datasets import load_dataset 


/home/iwaniukooo/Documents/Projects/wb2-llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset("go_emotions", "simplified")
with open("dict.txt", "r") as f:
    emotion_dict = [line.strip() for line in f]

emotion_dict

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [5]:


data_subset = dataset['train'].shuffle(seed=43).select(range(4500))  

all_texts = data_subset['text']
all_labels = data_subset['labels'] 
all_labels = [label[0] for label in all_labels]  
all_labels = [emotion_dict[int(label)] for label in all_labels]




In [6]:
n=14
n_plus=n+3

In [7]:
train_texts, val_texts = all_texts[:n], all_texts[n:n_plus]
train_labels, val_labels = all_labels[:n], all_labels[n:n_plus]

In [8]:
# llm = LLMModel()
# llm = LLMModel(model_name="huggyllama/llama-7b",)
# llm = LLMModel(model_name="facebook/opt-1.3B", verbose=True)
# llm = LLMModel(model_name="microsoft/phi-1.5")
# llm = LLMModel(model_name="EleutherAI/gpt-neo-2.7B",verbose=True)
# llm = LLMModel(model_name="EleutherAI/gpt-neox-20b",verbose=True)
llm = LLMModel(model_name="EleutherAI/gpt-neo-125m",verbose=True)


# llm = LLMModel(model_name="facebook/opt-iml-max-1.3b", verbose=True)





Loading model: EleutherAI/gpt-neo-125m
Model loaded successfully on cpu


In [9]:
llm.train(train_texts,train_labels)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Concept prediction prompt:
In 2 words guess, what task is the model doing, the format is INPUT, RESULT:
INPUT:Yeah for sure, they're too smart to go to class anyway, they've got time. 
RESULT:admiration

INPUT:Me too. I have so much shit and I can’t imagine taking any of it into my car and piling it that high. Wtf? 
RESULT:anger

INPUT:I get it. However that person doesn’t know who you are... 
RESULT:neutral

INPUT:He's both. An edgy neckbeard skinny fat, but he is undeniably funny.  
RESULT:amusement

INPUT:Keep in mind with these verses you've selected you aren't mentioning how we must he crucified in the flesh with [NAME]. 
RESULT:neutral

INPUT:I tried that. It doesn't work out very well. You become a machine. 
RESULT:disappointment

INPUT:I just want to say once you take action, stay safe!! Good luck. It’s the best to leave.  
RESULT:optimism

INPUT:There is nothing more statist than socialized and compulsory education. They subsist on violence and perpetuate violence. These peopl

In [10]:
llm.predict(val_texts[1])

Prediction prompt for 'I wasn’t expecting that tbh':
You are a classificator
 Your goal is to learn the concept from examples below:
INPUT:Yeah for sure, they're too smart to go to class anyway, they've got time. 
RESULT:admiration

INPUT:Me too. I have so much shit and I can’t imagine taking any of it into my car and piling it that high. Wtf? 
RESULT:anger

INPUT:I get it. However that person doesn’t know who you are... 
RESULT:neutral

INPUT:He's both. An edgy neckbeard skinny fat, but he is undeniably funny.  
RESULT:amusement

INPUT:Keep in mind with these verses you've selected you aren't mentioning how we must he crucified in the flesh with [NAME]. 
RESULT:neutral

INPUT:I tried that. It doesn't work out very well. You become a machine. 
RESULT:disappointment

INPUT:I just want to say once you take action, stay safe!! Good luck. It’s the best to leave.  
RESULT:optimism

INPUT:There is nothing more statist than socialized and compulsory education. They subsist on violence and per

'You are a classificator\n Your goal is to learn the concept from examples below:\nINPUT:Yeah for sure, they\'re too smart to go to class anyway, they\'ve got time. \nRESULT:admiration\n\nINPUT:Me too. I have so much shit and I can’t imagine taking any of it into my car and piling it that high. Wtf? \nRESULT:anger\n\nINPUT:I get it. However that person doesn’t know who you are... \nRESULT:neutral\n\nINPUT:He\'s both. An edgy neckbeard skinny fat, but he is undeniably funny.  \nRESULT:amusement\n\nINPUT:Keep in mind with these verses you\'ve selected you aren\'t mentioning how we must he crucified in the flesh with [NAME]. \nRESULT:neutral\n\nINPUT:I tried that. It doesn\'t work out very well. You become a machine. \nRESULT:disappointment\n\nINPUT:I just want to say once you take action, stay safe!! Good luck. It’s the best to leave.  \nRESULT:optimism\n\nINPUT:There is nothing more statist than socialized and compulsory education. They subsist on violence and perpetuate violence. These

In [11]:
from custom_model import EmotionClassifier
from custom_model import EmotionBERT
import torch

def main():
    
    classifier = EmotionClassifier(device='cpu')
    
    
    try:
        
        with open("dict.txt", "r") as f:
            emotion_labels = [line.strip() for line in f]
        
        
        classifier._setup_labels(emotion_labels)
        
        
        classifier.model = EmotionBERT(n_classes=len(emotion_labels))
        
        
        classifier.model.load_state_dict(torch.load('best_model_state.pth'))
        
        
        example_text = "i want to cry, depression is coming, life is worth nothing"
        predicted_labels = classifier.predict(example_text)
        
        print(f"Tekst: '{example_text}'")
        print(f"Przewidywane emocje: {predicted_labels}")
        
    except FileNotFoundError:
        print("Błąd: Nie znaleziono pliku modelu 'best_model_state.pth' lub 'dict.txt'")
    except Exception as e:
        print(f"Wystąpił błąd: {str(e)}")

if __name__ == "__main__":
    main()

Labels setup: 28 unique labels found: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']
Tekst: 'i want to cry, depression is coming, life is worth nothing'
Przewidywane emocje: {'sadness': 0.2733209431171417}


In [12]:
import re


In [ ]:
import random
import csv
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from basic.llm_model import LLMModel
from datasets import load_dataset
import re
from collections import defaultdict


def normalize_prediction(pred):
    """Normalize predictions by converting to lowercase, stripping, and removing special characters."""
    if pred is None:
        return ""  # Return an empty string for None values
    return re.sub(r"[.,']", "", pred.strip().lower())

# def compare_llm_classifier(all_texts, all_labels, n1, n2, random_seed, iteration, detailed_csv):
    """
    Trains LLM on classifier outputs and compares performance for one iteration.
    
    Args:
        all_texts: List of input texts
        all_labels: List of true labels
        n1: Number of training samples
        n2: Number of validation samples
        random_seed: Random seed for reproducibility
        iteration: Current iteration number
        detailed_csv: Output file for detailed results
    """
    random.seed(random_seed)
    
    # Initialize models
    classifier = initialize_classifier()
    llm = LLMModel(model_name="EleutherAI/gpt-neo-1.3B", verbose=True)
    
    # Sample training data
    train_indices = random.sample(range(len(all_texts)), n1)
    train_texts = [all_texts[idx] for idx in train_indices]
    train_labels = [all_labels[idx] for idx in train_indices]
    
    # Get classifier predictions on training data
    train_classifier_preds = []
    for text in train_texts:
        try:
            cls_output = classifier.predict(text)
            if isinstance(cls_output, dict):
                cls_pred = list(cls_output.keys())[0]  # Take just the key (emotion name)
            else:
                cls_pred = cls_output
            train_classifier_preds.append(cls_pred)
        except Exception as e:
            print(f"Error during classifier prediction: {e}")
            train_classifier_preds.append(None)
    
    # Train LLM on classifier predictions
    llm.train(train_texts, train_classifier_preds)
    
    # Sample validation data - ensure they're different from training data
    remaining_indices = list(set(range(len(all_texts))) - set(train_indices))
    val_indices = random.sample(remaining_indices, min(n2, len(remaining_indices)))
    val_texts = [all_texts[idx] for idx in val_indices]
    val_labels = [all_labels[idx] for idx in val_indices]
    
    # Make predictions on validation data
    detailed_results = []
    match_count = 0
    total_count = 0
    
    # Store all predictions for metrics calculation
    true_labels = []
    llm_predictions = []
    cls_predictions = []
    
    for text, true_label in zip(val_texts, val_labels):
        try:
            llm_pred = llm.predict(text)
        except Exception as e:
            print(f"Error during LLM prediction: {e}")
            llm_pred = None
        
        try:
            cls_output = classifier.predict(text)
            if isinstance(cls_output, dict):
                cls_pred = list(cls_output.keys())[0]
            else:
                cls_pred = cls_output
        except Exception as e:
            print(f"Error during classifier prediction: {e}")
            cls_pred = None
        
        # Check if predictions match after normalization
        llm_pred_norm = normalize_prediction(llm_pred)
        cls_pred_norm = normalize_prediction(cls_pred)
        is_match = llm_pred_norm == cls_pred_norm
        
        if llm_pred is not None and cls_pred is not None:
            total_count += 1
            if is_match:
                match_count += 1
            
            # Store for metrics calculation
            true_labels.append(true_label)
            llm_predictions.append(llm_pred)
            cls_predictions.append(cls_pred)
        
        # Store detailed results
        detailed_results.append({
            "Iteration": iteration,
            "Text": text,
            "True_Label": true_label,
            "Classifier_Prediction": cls_pred,
            "LLM_Prediction": llm_pred,
            "Match": str(is_match)  # Convert to string for CSV
        })
    
    # Calculate match rate and metrics for this iteration
    match_rate = match_count / total_count if total_count > 0 else 0
    
    # Calculate metrics (only if we have enough data)
    metrics = {}
    if len(true_labels) > 0:
        # Calculate accuracy
        llm_accuracy = accuracy_score(true_labels, llm_predictions)
        cls_accuracy = accuracy_score(true_labels, cls_predictions)
        
        try:
            # Calculate F1 score (micro average to handle multiclass)
            llm_f1 = f1_score(true_labels, llm_predictions, average='micro')
            cls_f1 = f1_score(true_labels, cls_predictions, average='micro')
            
            # Calculate precision and recall
            llm_precision = precision_score(true_labels, llm_predictions, average='micro')
            llm_recall = recall_score(true_labels, llm_predictions, average='micro')
            cls_precision = precision_score(true_labels, cls_predictions, average='micro')
            cls_recall = recall_score(true_labels, cls_predictions, average='micro')
            
            metrics = {
                "LLM_Accuracy": llm_accuracy,
                "CLS_Accuracy": cls_accuracy,
                "LLM_F1": llm_f1, 
                "CLS_F1": cls_f1,
                "LLM_Precision": llm_precision,
                "LLM_Recall": llm_recall,
                "CLS_Precision": cls_precision,
                "CLS_Recall": cls_recall
            }
        except Exception as e:
            print(f"Error calculating metrics: {e}")
            # Provide at least accuracy if other metrics fail
            metrics = {
                "LLM_Accuracy": llm_accuracy,
                "CLS_Accuracy": cls_accuracy
            }
    
    print(f"Iteration {iteration} match rate: {match_rate:.2f} ({match_count}/{total_count})")
    if metrics:
        print(f"LLM Accuracy: {metrics.get('LLM_Accuracy', 'N/A'):.4f}, F1: {metrics.get('LLM_F1', 'N/A'):.4f}")
        print(f"CLS Accuracy: {metrics.get('CLS_Accuracy', 'N/A'):.4f}, F1: {metrics.get('CLS_F1', 'N/A'):.4f}")
    
    # Save detailed results to CSV
    mode = "a" if iteration > 1 else "w"  # Append for iterations after the first
    with open(detailed_csv, mode=mode, newline="", encoding="utf-8") as csv_file:
        fieldnames = ["Iteration", "Text", "True_Label", "Classifier_Prediction", "LLM_Prediction", "Match"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        if iteration == 1:  # Write header only for the first iteration
            writer.writeheader()
        for row in detailed_results:
            writer.writerow(row)
    
    return match_rate, len(detailed_results), metrics
# def compare_llm_classifier(all_texts, all_labels, n1, n2, random_seed, iteration, detailed_csv):
    """
    Trains LLM on classifier outputs and compares performance for one iteration.
    
    Args:
        all_texts: List of input texts
        all_labels: List of true labels
        n1: Number of training samples
        n2: Number of validation samples
        random_seed: Random seed for reproducibility
        iteration: Current iteration number
        detailed_csv: Output file for detailed results
    """
    random.seed(random_seed)
    
    # Initialize models
    classifier = initialize_classifier()
    llm = LLMModel(model_name="EleutherAI/gpt-neo-1.3B", verbose=True)
    
    # Group indices by emotion to ensure at least one example per emotion
    emotion_to_indices = defaultdict(list)
    for i, emotion in enumerate(all_labels):
        emotion_to_indices[emotion].append(i)
    
    # Collect one sample from each available emotion first
    train_indices = []
    for emotion, indices in emotion_to_indices.items():
        if indices:
            # Randomly select one index for this emotion
            selected_idx = random.choice(indices)
            train_indices.append(selected_idx)
            # Remove the selected index to avoid duplicates
            indices.remove(selected_idx)
    
    # If we need more samples, randomly select from remaining data
    if len(train_indices) < n1:
        # Create a flat list of all remaining indices
        remaining_indices = []
        for indices in emotion_to_indices.values():
            remaining_indices.extend(indices)
        
        # Randomly select additional indices
        additional_count = min(n1 - len(train_indices), len(remaining_indices))
        if additional_count > 0:
            additional_indices = random.sample(remaining_indices, additional_count)
            train_indices.extend(additional_indices)
    
    # If we somehow have too many samples, trim the list
    if len(train_indices) > n1:
        train_indices = train_indices[:n1]
        
    print(f"Selected {len(train_indices)} training samples with {len(set([all_labels[idx] for idx in train_indices]))} different emotions")
    
    train_texts = [all_texts[idx] for idx in train_indices]
    train_labels = [all_labels[idx] for idx in train_indices]
    
    # Get classifier predictions on training data
    train_classifier_preds = []
    for text in train_texts:
        try:
            cls_output = classifier.predict(text)
            if isinstance(cls_output, dict):
                cls_pred = list(cls_output.keys())[0]  # Take just the key (emotion name)
            else:
                cls_pred = cls_output
            train_classifier_preds.append(cls_pred)
        except Exception as e:
            print(f"Error during classifier prediction: {e}")
            train_classifier_preds.append(None)
    
    # Print emotion distribution in training data
    emotion_counts = defaultdict(int)
    for pred in train_classifier_preds:
        if pred:
            emotion_counts[pred] += 1
    print("Classifier predictions distribution in training data:")
    for emotion, count in emotion_counts.items():
        print(f"  {emotion}: {count}")
    
    # Train LLM on classifier predictions
    llm.train(train_texts, train_classifier_preds)
    
    # Sample validation data - ensure they're different from training data
    remaining_indices = list(set(range(len(all_texts))) - set(train_indices))
    val_indices = random.sample(remaining_indices, min(n2, len(remaining_indices)))
    val_texts = [all_texts[idx] for idx in val_indices]
    val_labels = [all_labels[idx] for idx in val_indices]
    
    # Make predictions on validation data
    detailed_results = []
    match_count = 0
    total_count = 0
    
    # Store all predictions for metrics calculation
    true_labels = []
    llm_predictions = []
    cls_predictions = []
    
    for text, true_label in zip(val_texts, val_labels):
        try:
            llm_pred = llm.predict(text)
        except Exception as e:
            print(f"Error during LLM prediction: {e}")
            llm_pred = None
        
        try:
            cls_output = classifier.predict(text)
            if isinstance(cls_output, dict):
                cls_pred = list(cls_output.keys())[0]
            else:
                cls_pred = cls_output
        except Exception as e:
            print(f"Error during classifier prediction: {e}")
            cls_pred = None
        
        # Check if predictions match after normalization
        llm_pred_norm = normalize_prediction(llm_pred)
        cls_pred_norm = normalize_prediction(cls_pred)
        is_match = llm_pred_norm == cls_pred_norm
        
        if llm_pred is not None and cls_pred is not None:
            total_count += 1
            if is_match:
                match_count += 1
            
            # Store for metrics calculation
            true_labels.append(true_label)
            llm_predictions.append(llm_pred)
            cls_predictions.append(cls_pred)
        
        # Store detailed results
        detailed_results.append({
            "Iteration": iteration,
            "Text": text,
            "True_Label": true_label,
            "Classifier_Prediction": cls_pred,
            "LLM_Prediction": llm_pred,
            "Match": str(is_match)  # Convert to string for CSV
        })
    
    # Calculate match rate and metrics for this iteration
    match_rate = match_count / total_count if total_count > 0 else 0
    
    # Calculate metrics (only if we have enough data)
    metrics = {}
    if len(true_labels) > 0:
        # Calculate accuracy
        llm_accuracy = accuracy_score(true_labels, llm_predictions)
        cls_accuracy = accuracy_score(true_labels, cls_predictions)
        
        try:
            # Calculate F1 score (micro average to handle multiclass)
            llm_f1 = f1_score(true_labels, llm_predictions, average='micro')
            cls_f1 = f1_score(true_labels, cls_predictions, average='micro')
            
            # Calculate precision and recall
            llm_precision = precision_score(true_labels, llm_predictions, average='micro')
            llm_recall = recall_score(true_labels, llm_predictions, average='micro')
            cls_precision = precision_score(true_labels, cls_predictions, average='micro')
            cls_recall = recall_score(true_labels, cls_predictions, average='micro')
            
            metrics = {
                "LLM_Accuracy": llm_accuracy,
                "CLS_Accuracy": cls_accuracy,
                "LLM_F1": llm_f1, 
                "CLS_F1": cls_f1,
                "LLM_Precision": llm_precision,
                "LLM_Recall": llm_recall,
                "CLS_Precision": cls_precision,
                "CLS_Recall": cls_recall
            }
        except Exception as e:
            print(f"Error calculating metrics: {e}")
            # Provide at least accuracy if other metrics fail
            metrics = {
                "LLM_Accuracy": llm_accuracy,
                "CLS_Accuracy": cls_accuracy
            }
    
    print(f"Iteration {iteration} match rate: {match_rate:.2f} ({match_count}/{total_count})")
    if metrics:
        print(f"LLM Accuracy: {metrics.get('LLM_Accuracy', 'N/A'):.4f}, F1: {metrics.get('LLM_F1', 'N/A'):.4f}")
        print(f"CLS Accuracy: {metrics.get('CLS_Accuracy', 'N/A'):.4f}, F1: {metrics.get('CLS_F1', 'N/A'):.4f}")
    
    # Save detailed results to CSV
    mode = "a" if iteration > 1 else "w"  # Append for iterations after the first
    with open(detailed_csv, mode=mode, newline="", encoding="utf-8") as csv_file:
        fieldnames = ["Iteration", "Text", "True_Label", "Classifier_Prediction", "LLM_Prediction", "Match"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        if iteration == 1:  # Write header only for the first iteration
            writer.writeheader()
        for row in detailed_results:
            writer.writerow(row)
    
    return match_rate, len(detailed_results), metrics

def compare_llm_classifier(all_texts, all_labels, n1, n2, random_seed, iteration, detailed_csv):
    """
    Trains LLM on classifier outputs and compares performance for one iteration.
    
    Args:
        all_texts: List of input texts
        all_labels: List of true labels
        n1: Number of training samples
        n2: Number of validation samples
        random_seed: Random seed for reproducibility
        iteration: Current iteration number
        detailed_csv: Output file for detailed results
    """
    random.seed(random_seed)
    
    # Initialize models
    classifier = initialize_classifier()
    llm = LLMModel(model_name="EleutherAI/gpt-neo-2.7B", verbose=True)
    
    # Group indices by emotion to ensure at least one example per emotion
    emotion_to_indices = defaultdict(list)
    for i, emotion in enumerate(all_labels):
        emotion_to_indices[emotion].append(i)
    
    # Limit neutral samples to 2 max
    if "neutral" in emotion_to_indices and len(emotion_to_indices["neutral"]) > 2:
        emotion_to_indices["neutral"] = random.sample(emotion_to_indices["neutral"], 2)
        print("Limited neutral samples to 2")
    
    # Collect one sample from each available emotion first
    train_indices = []
    emotions_used = set()
    
    # Prioritize non-neutral emotions
    emotion_list = sorted(emotion_to_indices.keys(), key=lambda x: x == "neutral")
    
    for emotion in emotion_list:
        indices = emotion_to_indices[emotion]
        if indices:
            # Randomly select one index for this emotion
            selected_idx = random.choice(indices)
            train_indices.append(selected_idx)
            emotions_used.add(emotion)
            # Remove the selected index to avoid duplicates
            indices.remove(selected_idx)
    
    # If we need more samples, randomly select from remaining data
    if len(train_indices) < n1:
        # Create a flat list of all remaining indices (except neutral if we already have 2)
        remaining_indices = []
        for emotion, indices in emotion_to_indices.items():
            if emotion != "neutral" or emotion_to_indices["neutral"].count("neutral") < 2:
                remaining_indices.extend(indices)
        
        # Randomly select additional indices
        if remaining_indices:
            additional_count = min(n1 - len(train_indices), len(remaining_indices))
            if additional_count > 0:
                additional_indices = random.sample(remaining_indices, additional_count)
                train_indices.extend(additional_indices)
    
    # If we somehow have too many samples, trim the list
    if len(train_indices) > n1:
        train_indices = train_indices[:n1]
        
    # Get training texts and labels
    train_texts = [all_texts[idx] for idx in train_indices]
    train_labels = [all_labels[idx] for idx in train_indices]
    
    # Print emotion distribution in training data
    emotion_counts = defaultdict(int)
    for label in train_labels:
        emotion_counts[label] += 1
        
    print(f"Selected {len(train_indices)} training samples with {len(emotions_used)} different emotions")
    print("Emotion distribution in training data:")
    for emotion, count in emotion_counts.items():
        print(f"  {emotion}: {count}")
    
    # Get classifier predictions on training data
    train_classifier_preds = []
    for text in train_texts:
        try:
            cls_output = classifier.predict(text)
            if isinstance(cls_output, dict):
                cls_pred = list(cls_output.keys())[0]  # Take just the key (emotion name)
            else:
                cls_pred = cls_output
            train_classifier_preds.append(cls_pred)
        except Exception as e:
            print(f"Error during classifier prediction: {e}")
            train_classifier_preds.append(None)
    
    # Print classifier prediction distribution
    pred_counts = defaultdict(int)
    for pred in train_classifier_preds:
        if pred:
            pred_counts[pred] += 1
    print("Classifier predictions distribution in training data:")
    for emotion, count in pred_counts.items():
        print(f"  {emotion}: {count}")
    
    # Train LLM on classifier predictions
    llm.train(train_texts, train_classifier_preds)
    
    # Sample validation data - ensure they're different from training data
    remaining_indices = list(set(range(len(all_texts))) - set(train_indices))
    val_indices = random.sample(remaining_indices, min(n2, len(remaining_indices)))
    val_texts = [all_texts[idx] for idx in val_indices]
    val_labels = [all_labels[idx] for idx in val_indices]
    
    # Make predictions on validation data
    detailed_results = []
    match_count = 0
    total_count = 0
    
    # Store all predictions for metrics calculation
    true_labels = []
    llm_predictions = []
    cls_predictions = []
    
    for text, true_label in zip(val_texts, val_labels):
        try:
            llm_pred = llm.predict(text)
        except Exception as e:
            print(f"Error during LLM prediction: {e}")
            llm_pred = None
        
        try:
            cls_output = classifier.predict(text)
            if isinstance(cls_output, dict):
                cls_pred = list(cls_output.keys())[0]
            else:
                cls_pred = cls_output
        except Exception as e:
            print(f"Error during classifier prediction: {e}")
            cls_pred = None
        
        # Check if predictions match after normalization
        llm_pred_norm = normalize_prediction(llm_pred)
        cls_pred_norm = normalize_prediction(cls_pred)
        is_match = llm_pred_norm == cls_pred_norm
        
        if llm_pred is not None and cls_pred is not None:
            total_count += 1
            if is_match:
                match_count += 1
            
            # Store for metrics calculation
            true_labels.append(true_label)
            llm_predictions.append(llm_pred)
            cls_predictions.append(cls_pred)
        
        # Store detailed results
        detailed_results.append({
            "Iteration": iteration,
            "Text": text,
            "True_Label": true_label,
            "Classifier_Prediction": cls_pred,
            "LLM_Prediction": llm_pred,
            "Match": str(is_match)  # Convert to string for CSV
        })
    
    # Calculate match rate and metrics for this iteration
    match_rate = match_count / total_count if total_count > 0 else 0
    
    # Calculate metrics (only if we have enough data)
    metrics = {}
    if len(true_labels) > 0:
        # Calculate accuracy
        llm_accuracy = accuracy_score(true_labels, llm_predictions)
        cls_accuracy = accuracy_score(true_labels, cls_predictions)
        
        try:
            # Calculate F1 score (micro average to handle multiclass)
            llm_f1 = f1_score(true_labels, llm_predictions, average='micro')
            cls_f1 = f1_score(true_labels, cls_predictions, average='micro')
            
            # Calculate precision and recall
            llm_precision = precision_score(true_labels, llm_predictions, average='micro')
            llm_recall = recall_score(true_labels, llm_predictions, average='micro')
            cls_precision = precision_score(true_labels, cls_predictions, average='micro')
            cls_recall = recall_score(true_labels, cls_predictions, average='micro')
            
            metrics = {
                "LLM_Accuracy": llm_accuracy,
                "CLS_Accuracy": cls_accuracy,
                "LLM_F1": llm_f1, 
                "CLS_F1": cls_f1,
                "LLM_Precision": llm_precision,
                "LLM_Recall": llm_recall,
                "CLS_Precision": cls_precision,
                "CLS_Recall": cls_recall
            }
        except Exception as e:
            print(f"Error calculating metrics: {e}")
            # Provide at least accuracy if other metrics fail
            metrics = {
                "LLM_Accuracy": llm_accuracy,
                "CLS_Accuracy": cls_accuracy
            }
    
    print(f"Iteration {iteration} match rate: {match_rate:.2f} ({match_count}/{total_count})")
    if metrics:
        print(f"LLM Accuracy: {metrics.get('LLM_Accuracy', 'N/A'):.4f}, F1: {metrics.get('LLM_F1', 'N/A'):.4f}")
        print(f"CLS Accuracy: {metrics.get('CLS_Accuracy', 'N/A'):.4f}, F1: {metrics.get('CLS_F1', 'N/A'):.4f}")
    
    # Save detailed results to CSV
    mode = "a" if iteration > 1 else "w"  # Append for iterations after the first
    with open(detailed_csv, mode=mode, newline="", encoding="utf-8") as csv_file:
        fieldnames = ["Iteration", "Text", "True_Label", "Classifier_Prediction", "LLM_Prediction", "Match"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        if iteration == 1:  # Write header only for the first iteration
            writer.writeheader()
        for row in detailed_results:
            writer.writerow(row)
    
    return match_rate, len(detailed_results), metrics

def run_experiment(all_texts, all_labels, n1, n2, k, random_seed, detailed_csv, summary_csv="summary_results.csv"):
    """
    Runs the comparison experiment k times and aggregates results.
    
    Args:
        all_texts: List of input texts
        all_labels: List of true labels
        n1: Number of training samples
        n2: Number of validation samples
        k: Number of iterations
        random_seed: Random seed for reproducibility
        detailed_csv: Output file for detailed results
        summary_csv: Output file for summary results
    """
    # Delete existing CSV files to avoid appending to old data
    if os.path.exists(detailed_csv):
        os.remove(detailed_csv)
    if os.path.exists(summary_csv):
        os.remove(summary_csv)
    
    # Run k iterations
    summary_results = []
    total_rows = 0
    
    for i in range(1, k + 1):
        print(f"\nRunning iteration {i}/{k}...")
        match_rate, rows_added, metrics = compare_llm_classifier(all_texts, all_labels, n1, n2, random_seed + i, i, detailed_csv)
        total_rows += rows_added
        
        # Add metrics to summary results
        result = {
            "Iteration": i,
            "Training_Size": n1,
            "Validation_Size": n2,
            "Match_Rate": match_rate
        }
        # Add any available metrics
        result.update(metrics)
        summary_results.append(result)
    
    # Save summary results with metrics
    with open(summary_csv, mode="w", newline="", encoding="utf-8") as csv_file:
        # Determine fieldnames including all possible metrics
        fieldnames = ["Iteration", "Training_Size", "Validation_Size", "Match_Rate"]
        for row in summary_results:
            for key in row:
                if key not in fieldnames:
                    fieldnames.append(key)
        
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for row in summary_results:
            writer.writerow(row)
    
    # Calculate overall match rate and average metrics
    overall_match_rate = sum(row["Match_Rate"] for row in summary_results) / len(summary_results)
    
    # Calculate average values for each metric across iterations
    avg_metrics = {}
    for key in fieldnames:
        if key not in ["Iteration", "Training_Size", "Validation_Size"]:
            values = [row.get(key, 0) for row in summary_results if key in row]
            if values:
                avg_metrics[f"Avg_{key}"] = sum(values) / len(values)
    
    print(f"\nExperiment completed:")
    print(f"- Detailed results saved to {detailed_csv} ({total_rows} rows)")
    print(f"- Summary results saved to {summary_csv}")
    print(f"- Overall match rate: {overall_match_rate:.4f}")
    print(f"- Expected rows: {k * n2}, Actual rows: {total_rows}")
    
    # Print average metrics
    for metric, value in avg_metrics.items():
        print(f"- {metric}: {value:.4f}")

def initialize_classifier():
    """Initialize and return the emotion classifier."""
    try:
        from custom_model import EmotionClassifier, EmotionBERT
        import torch
        
        classifier = EmotionClassifier(device="cpu")
        with open("dict.txt", "r") as f:
            emotion_labels = [line.strip() for line in f]
        classifier._setup_labels(emotion_labels)
        classifier.model = EmotionBERT(n_classes=len(emotion_labels))
        classifier.model.load_state_dict(torch.load("best_model_state.pth"))
        return classifier
    except Exception as e:
        print(f"Failed to load classifier: {e}")
        return None

if __name__ == "__main__":
    # Load data
    dataset = load_dataset("go_emotions", "simplified")
    with open("dict.txt", "r") as f:
        emotion_dict = [line.strip() for line in f]
    data_subset = dataset["train"].shuffle(seed=43).select(range(4500))
    all_texts = data_subset["text"]
    all_labels = data_subset["labels"]
    all_labels = [label[0] for label in all_labels]
    all_labels = [emotion_dict[int(label)] for label in all_labels]
    
    # Run experiment
    run_experiment(all_texts, all_labels, n1=60, n2=240, k=1, random_seed=42, detailed_csv="detailed_results.csv")


Running iteration 1/1...
Labels setup: 28 unique labels found: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']
Loading model: EleutherAI/gpt-neo-2.7B
Model loaded successfully on cpu
Limited neutral samples to 2
Selected 60 training samples with 28 different emotions
Emotion distribution in training data:
  admiration: 8
  anger: 2
  amusement: 4
  disappointment: 1
  optimism: 1
  joy: 3
  desire: 1
  annoyance: 2
  embarrassment: 1
  realization: 1
  gratitude: 3
  confusion: 1
  disapproval: 2
  surprise: 3
  approval: 3
  love: 5
  curiosity: 6
  disgust: 2
  caring: 1
  sadness: 1
  excitement: 1
  remorse: 1
  relief: 1
  fear: 1
  grief: 1
  nervousness: 2
  pride: 1
  neutral: 1
Classifier predic